In [75]:
!nvidia-smi | head

Mon Nov  4 13:09:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0              27W /  70W |   7641MiB / 15360MiB |      0%      Default |


In [76]:
#Installing the Polars GPU Engine
!pip install -U polars[gpu] --extra-index-url=https://pypi.nvidia.com

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
  Using cached polars-1.12.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (14 kB)
Using cached polars-1.12.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.1 MB)
  Attempting uninstall: polars
    Found existing installation: polars 1.7.0
    Uninstalling polars-1.7.0:
      Successfully uninstalled polars-1.7.0


In [77]:
#Install Other Important Dependencies
!pip install hvplot jupyter_bokeh holoviews==1.19 pynvml

In [78]:
!pip install polars==1.7

  Using cached polars-1.7.0-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (14 kB)
Reason for being yanked: prefiltered strategy is wrong
Using cached polars-1.7.0-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (32.2 MB)
  Attempting uninstall: polars
    Found existing installation: polars 1.12.0
    Uninstalling polars-1.12.0:
      Successfully uninstalled polars-1.12.0


In [79]:
#Download the Data
# We'll be working with a roughly 22GB dataset of simulated financial transactions from Kaggle by default. If you're running this notebook
# on Google Colab using the T4 GPU in the Colab free tier, we'll download a smaller version of this dataset (about 20% of the size) to fit on the relatively weaker CPU and GPU.

import pynvml
pynvml.nvmlInit()
pynvml.nvmlDeviceGetName(pynvml.nvmlDeviceGetHandleByIndex(0))
mem = pynvml.nvmlDeviceGetMemoryInfo(pynvml.nvmlDeviceGetHandleByIndex(0))
mem = mem.total/1e9
if mem < 24:
    !wget https://storage.googleapis.com/rapidsai/polars-demo/transactions-t4-20.parquet -O transactions.parquet
else:
    !wget https://storage.googleapis.com/rapidsai/polars-demo/transactions.parquet -O transactions.parquet

!wget https://storage.googleapis.com/rapidsai/polars-demo/rainfall_data_2010_2020.csv

--2024-11-04 13:10:00--  https://storage.googleapis.com/rapidsai/polars-demo/transactions-t4-20.parquet
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.207, 74.125.135.207, 142.250.99.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 857658422 (818M) [application/octet-stream]
Saving to: ‘transactions.parquet’

transactions.parque 100%[===================>] 817.93M  96.1MB/s    in 6.9s    

2024-11-04 13:10:07 (119 MB/s) - ‘transactions.parquet’ saved [857658422/857658422]

--2024-11-04 13:10:07--  https://storage.googleapis.com/rapidsai/polars-demo/rainfall_data_2010_2020.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.207, 74.125.135.207, 142.250.99.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131421 (128K) [tex

In [80]:
import polars as pl
import hvplot.polars # Using the hvplot as the plotting backend
from polars.testing import assert_frame_equal

In [81]:
pl.__version__ # Make sure the version is >= `1.6.0`; If not restart the session.

'1.7.0'

In [82]:
transactions = pl.scan_parquet("transactions.parquet")

In [83]:
transactions.collect_schema()

Schema([('CUST_ID', String),
        ('START_DATE', Date),
        ('END_DATE', Date),
        ('TRANS_ID', String),
        ('DATE', Date),
        ('YEAR', Int64),
        ('MONTH', Int64),
        ('DAY', Int64),
        ('EXP_TYPE', String),
        ('AMOUNT', Float64)])

In [84]:
transactions.head(5).collect()

CUST_ID,START_DATE,END_DATE,TRANS_ID,DATE,YEAR,MONTH,DAY,EXP_TYPE,AMOUNT
str,date,date,str,date,i64,i64,i64,str,f64
"""CI6XLYUMQK""",2015-05-01,null,"""T8I9ZB5A6X90UG8""",2015-09-11,2015,9,11,"""Motor/Travel""",20.27
"""CI6XLYUMQK""",2015-05-01,null,"""TZ4JSLS7SC7FO9H""",2017-02-08,2017,2,8,"""Motor/Travel""",12.85
"""CI6XLYUMQK""",2015-05-01,null,"""TTUKRDDJ6B6F42H""",2015-08-01,2015,8,1,"""Housing""",383.8
"""CI6XLYUMQK""",2015-05-01,null,"""TDUHFRUKGPPI6HD""",2019-03-16,2019,3,16,"""Entertainment""",5.72
"""CI6XLYUMQK""",2015-05-01,null,"""T0JBZHBMSVRFMMD""",2015-05-15,2015,5,15,"""Entertainment""",11.06


In [85]:
#First GRU Operation: Total aggregate transaction amount
%%time
transactions.select(pl.col("AMOUNT").sum()).collect()

CPU times: user 1.55 s, sys: 102 ms, total: 1.65 s
Wall time: 1.31 s


AMOUNT
f64
3.6183e9


In [86]:
%%time
transactions.select(pl.col("AMOUNT").sum()).collect(engine = "gpu")

CPU times: user 276 ms, sys: 31.6 ms, total: 308 ms
Wall time: 301 ms


AMOUNT
f64
3.6183e9


In [87]:
gpu_engine = pl.GPUEngine(
    device = 0, #This is default
    raise_on_fail=True,  #Fail loudly if we can't run on the GPU
)

In [88]:
%%time
transactions.select(pl.col("AMOUNT").sum()).collect(engine = "gpu")

CPU times: user 136 ms, sys: 31.5 ms, total: 168 ms
Wall time: 187 ms


AMOUNT
f64
3.6183e9


In [89]:
#Which customers have the largest total transactions?
#with CPU

%%time

res_cpu = (
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").sum())
    .sort(by="AMOUNT", descending=True)
    .head()
    .collect()
)
res_cpu

CPU times: user 4.34 s, sys: 3.29 s, total: 7.63 s
Wall time: 8.18 s


CUST_ID,AMOUNT
str,f64
"""CA9UYOQ5DA""",2.0290e6
"""CJUK2MTM5Q""",1.8115e6
"""CYXX1NBIKL""",1.8082e6
"""C6ILEYAYQ9""",1.7961e6
"""CCNBC305GI""",1.7274e6


In [90]:
#Which customers have the largest total transactions?
#with GPU

%%time

res_gpu = (
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").sum())
    .sort(by="AMOUNT", descending=True)
    .head()
    .collect(engine=gpu_engine)
)
res_gpu

CPU times: user 312 ms, sys: 54.7 ms, total: 367 ms
Wall time: 343 ms


CUST_ID,AMOUNT
str,f64
"""CA9UYOQ5DA""",2.0290e6
"""CJUK2MTM5Q""",1.8115e6
"""CYXX1NBIKL""",1.8082e6
"""C6ILEYAYQ9""",1.7961e6
"""CCNBC305GI""",1.7274e6


In [91]:
assert_frame_equal(res_cpu, res_gpu)

In [92]:
#In addition to the Dataframe interface, Polars also has an SQL interface.
#We can also use this with the GPU engine, since Polars translates both the DataFrame and SQL interfaces into a query execution plan.

#Polars also have SQL interface
query = """
SELECT CUST_ID, SUM(AMOUNT) as sum_amt
FROM transactions
GROUP BY CUST_ID
ORDER BY sum_amt desc
LIMIT 5
"""

%time pl.sql(query).collect()
%time pl.sql(query).collect(engine=gpu_engine)

CPU times: user 4.28 s, sys: 2.98 s, total: 7.26 s
Wall time: 5.6 s
CPU times: user 325 ms, sys: 27.5 ms, total: 352 ms
Wall time: 325 ms


CUST_ID,sum_amt
str,f64
"""CA9UYOQ5DA""",2.0290e6
"""CJUK2MTM5Q""",1.8115e6
"""CYXX1NBIKL""",1.8082e6
"""C6ILEYAYQ9""",1.7961e6
"""CCNBC305GI""",1.7274e6


In [93]:
#Which customers have the largest single transaction?
%%time

(
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").max().alias("max_amount"))
    .sort(by="max_amount", descending=True)
    .head()
    .collect()
)

CPU times: user 4.36 s, sys: 2.62 s, total: 6.98 s
Wall time: 4.54 s


CUST_ID,max_amount
str,f64
"""CIP0I11MG2""",6201.45
"""C4O38N5TQS""",6077.49
"""CL2M3N3K90""",6041.59
"""CC472PU9O8""",5929.14
"""CGR8UI27OK""",5903.61


In [94]:
#Which customers have the largest single transaction?
%%time

(
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").max().alias("max_amount"))
    .sort(by="max_amount", descending=True)
    .head()
    .collect(engine=gpu_engine)
)

CPU times: user 366 ms, sys: 31.5 ms, total: 397 ms
Wall time: 370 ms


CUST_ID,max_amount
str,f64
"""CIP0I11MG2""",6201.45
"""C4O38N5TQS""",6077.49
"""CL2M3N3K90""",6041.59
"""CC472PU9O8""",5929.14
"""CGR8UI27OK""",5903.61


In [95]:
#Let's bring in some weather data to evaluate if transactions are impacted by weather. For example, spending behavior for certain categories could be influenced by whether it's raining or even how much it's raining.

#Like our transactions dataset, here we'll use a simulated rainfall dataset.


names = ['Location', 'Rainfall (inches)', 'Date', 'YEAR', 'MONTH', 'DAY']

weather = pl.scan_csv("rainfall_data_2010_2020.csv", new_columns=names)
weather.head().collect()

Location,Rainfall (inches),Date,YEAR,MONTH,DAY
str,f64,i64,i64,i64,i64
"""Tatooine""",0.33,20100101,2010,1,1
"""Tatooine""",0.0,20100102,2010,1,2
"""Tatooine""",0.28,20100103,2010,1,3
"""Tatooine""",0.26,20100104,2010,1,4
"""Tatooine""",0.39,20100105,2010,1,5


In [96]:
weather_cleaned = (
    weather
    .with_columns(pl.col("Date").cast(pl.Utf8).str.strptime(pl.Date(), "%Y%m%d"))
    .collect()
)

In [97]:
#Is more rain related to more transactions across specific categories?

%%time

(
    transactions
    .join(
        other=weather_cleaned.lazy(),
        left_on="DATE",
        right_on="Date",
        how="inner"
    )
    .group_by(["EXP_TYPE", "DATE"])
    .agg(pl.mean("Rainfall (inches)"))
    .sort(["DATE", "EXP_TYPE", "Rainfall (inches)"])
    .head()
    .collect()
)

CPU times: user 13.5 s, sys: 3.94 s, total: 17.5 s
Wall time: 11.7 s


EXP_TYPE,DATE,Rainfall (inches)
str,date,f64
"""Bills and Utilities""",2010-01-01,0.33
"""Clothing""",2010-01-01,0.33
"""Education""",2010-01-01,0.33
"""Entertainment""",2010-01-01,0.33
"""Fines""",2010-01-01,0.33


In [98]:
%%time
res = (
    transactions
    .join(
        other=weather_cleaned.lazy(),
        left_on="DATE",
        right_on="Date",
        how="inner"
    )
    .group_by(["EXP_TYPE", "DATE"])
    .agg(pl.mean("Rainfall (inches)"))
    .sort(["DATE", "EXP_TYPE", "Rainfall (inches)"])
    .head()
    .collect(engine=gpu_engine)
)

res

CPU times: user 406 ms, sys: 13.1 ms, total: 419 ms
Wall time: 396 ms


EXP_TYPE,DATE,Rainfall (inches)
str,date,f64
"""Bills and Utilities""",2010-01-01,0.33
"""Clothing""",2010-01-01,0.33
"""Education""",2010-01-01,0.33
"""Entertainment""",2010-01-01,0.33
"""Fines""",2010-01-01,0.33
